In [1]:
import sys
#only need to install once
#!{sys.executable} -m pip install librosa

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter,filtfilt
import librosa
import librosa.display

import torch
import torchvision
from torch.autograd import Variable
from torchvision.io import read_image
import torch.nn.functional as F
import matplotlib.pyplot as plt
import cv2

In [2]:
#low-pass filtering function
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [3]:
def predict_image(image, model):
    image_tensor = image.float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = F.sigmoid(output).data.cpu().numpy()
    return index

In [4]:
device = torch.device('cuda')
print("GPU name: ", torch.cuda.get_device_name(0))
resnet18 = torchvision.models.resnet18(pretrained=False)
resnet18.fc = torch.nn.Linear(512, 1) #just 1 output, will use sigmoid later
PATH = './e6992/patient3-8.pth' #load the saved weights
resnet18.load_state_dict(torch.load(PATH))
resnet18.eval() #set eval mode
resnet18.to(device)

GPU name:  NVIDIA Tegra X1


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
Fs, samples = wavfile.read('./e6992/00001010-100507.wav')
total_seconds = int(len(samples)/Fs)

for second in range(1666, 1667, 1): #go through every second rolling window
    
    x = samples[(second)*Fs:(second+60)*Fs] #take out the selected minute

    cutoff = 1000 #define filter cutoff frequency (1000Hz)
    order = 5 #define filter order
    x_filtered = butter_lowpass_filter(x, cutoff, Fs, order) #low-pass filtering

    x_downsampled = signal.resample(x_filtered, 2048*60) #downsample the function to 2kHz
    new_Fs = 2048
    
    n_fft = 512 #length of fft, 512 optimal for speech

    sgram = librosa.stft(x_downsampled, center=False, n_fft=n_fft)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=new_Fs, n_fft=n_fft)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
    #for raw graphs to save
    #librosa.display.specshow(mel_sgram, sr=Fs, n_fft=n_fft, fmin=22, fmax=700)
    #for graphs with axis and scale
    librosa.display.specshow(mel_sgram, sr=new_Fs*4, n_fft=n_fft, fmin=22, fmax=700)
    #plt.colorbar(format='%+2.0f dB')
    file = '00000995-100507' + '_0' + str(second) + '.jpg'
    #print(file)
    plt.savefig(file) #need to save the figure as a jpg
    plt.close()
    img = cv2.imread(file) #re-open the file to crop off white space
    img = img[34:253, 53:390]
    cv2.imwrite(file, img)

In [6]:
img_path = '00000995-100507_01666.jpg'
image = read_image(img_path)
print(image.shape)

#display selected image, commented out for timing
#img = image.permute(1,2,0)
#plt.imshow(img)
#plt.show()

result = predict_image(image, resnet18)[0][0]
print(result)
print("0=healthy 1=apnea - result: ", round(result))

torch.Size([3, 219, 337])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0.9976793
0=healthy 1=apnea - result:  1
